## FRIB-TA Hands-on: Error mitigation [Day 3 morning]

Helpful links:

- [Lecture plan](https://github.com/NuclearPhysicsWorkshops/FRIB-TASummerSchoolQuantumComputing#detailed-lecture-plan)

## Goal

The goal of this notebook is to implement zero-noise extrapolation (ZNE) and use it to improve the performance of a simple quantum computation. By doing so, you will be able to implement ZNE in your own computations, both "from scratch" and by using existing software.

## Setup

In [ ]:
try:
    import qiskit
except ImportError:
    !pip install qiskit~=0.20.0 pylatexenc --quiet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# To ignore circuit addition deprecation warnings in Qiskit, which we'll use here.
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import qiskit
import qiskit.test.mock

version_requires = "0.20"
assert version_requires in qiskit.__version__, print(f"Qiskit version should be >={version_requires} but is {qiskit.__version__}")

## The quantum computation

Let us evaluate the probability of the ground state in a circuit which logically compiles to identity. Throughout, let us call the circuit $|\psi\rangle$ and use $\Pi := |0\rangle \langle 0|$ so that the probability of the ground state is $\langle \psi | \Pi | \psi \rangle$. 

In the cell below, write a function which returns a single-qubit quantum circuit with a specified (even) number of `X` gates.

In [ ]:
"""Set up the quantum computation to use with ZNE."""
def get_circuit(depth: int = 50) -> qiskit.QuantumCircuit:
    # -->  Your code here.
    pass

Let's visualize the circuit.

In [ ]:
"""Visualize the circuit."""
circuit = get_circuit()
circuit.draw()

## The noisy result

It's possible to run this notebook on real devices if you have access to them, but that would take a while due to the fair-share queue. Thankfully, the `qiskit.test.mock` module contains mock backends which model the noise characteristics of various devices. The next few cells show an example of how to use this.

In [ ]:
"""Getting a noisy device via qiskit.test.mock."""
# Get a noisy backend.
noisy_backend = qiskit.test.mock.FakeArmonk()

# You can see some of its properties as follows.
print("Basis gates:", noisy_backend.configuration().basis_gates)
print("Number of qubits:", noisy_backend.configuration().num_qubits)

If you'd like, you can see "lower-level" properties with `noisy_backend.configuration().hamiltonian`. To run on this device, we use it with `qiskit.execute` as usual.

In [ ]:
"""Running on a noisy device via qiskit.test.mock."""
job = qiskit.execute(circuit, backend=noisy_backend, optimization_level=0, shots=10_000)

Notes:
- If you're following along as intended, the circuit run in that cell doesn't have any measurements, so you'll get an error if you try to access them. 
- We set `optimization_level=0` which says, inasmuch as possible, to run exactly the circuit we passed in. This will be essential to run zero-noise extrapolation because this technique adds in extra gates which compile to identity.

With ZNE, we need to run multiple circuits, so it's convenient to have a function for this. Complete the body of the function in the cell below by:

1. Measuring the qubit in the circuit if `measure` is `True`. You'll want to make a copy of the circuit so that you don't modify the input circuit.
2. Calling `qiskit.execute` with the circuit, backend, and shots. **Be sure to set `optimization_level=0`**.
3. Returning the ratio of `0`s measured to the total number of shots.

In [ ]:
def execute(
    circuit: qiskit.QuantumCircuit, 
    backend: "qiskit.Sampler" = qiskit.test.mock.FakeArmonk(), 
    shots: int = 10_000,
    measure: bool = True,
) -> float:
    """Returns the ground state probability of the circuit run on the backend.

    Args:
        circuit: Single-qubit circuit to estimate the ground state probability of.
        backend: Backend to run the circuit on.
        shots: Number of times to sample from the circuit.
        measure: If True, measure all qubits in the circuit, else do nothing.
    """
    # -->  Your code here.
    return 0.0

Using this function, we can run the quantum computation and see our result.

In [ ]:
noisy_value = execute(circuit)
print("Noisy: \t⟨𝜓|Π|𝜓⟩ =", noisy_value)

## The true (noiseless) result

For testing how well ZNE works, it's convenient to know what the "true" or noiseless expectation value is. We already know the true result for our simple example, but we can do this in general for simple cases by using our function with a noiseless simulator as below.

In [ ]:
"""Determine the true (noiseless) value by running on a noiseless backend."""
true_value = execute(circuit, qiskit.Aer.get_backend("qasm_simulator"))
print("True: \t⟨𝜓|Π|𝜓⟩ =", true_value)

## Implementing zero-noise extrapolation

Now we implement ZNE in three steps.

### Step 1 / 3: Scale the noise in the computation

First we need to scale the noise in the computation. As described in lecture, there are various ways to do this. We will use the simplest gate-level method to scale noise. Letting $U$ denote the complete circuit (without measurements), map $U$ to $U U^\dagger U$. The next cell shows how to do this in Qiskit.

In [ ]:
"""Map U to U U^dag U."""
inverse = circuit.inverse()
scaled_circuit = circuit + inverse + circuit

You may wish to print / inspect this circuit.

For ZNE, we want to run at several noise levels. This corresponds to the mapping $U \mapsto U (U^\dagger U)^k$ for $k = 1, 2, 3, ...$. In the next cell, you are asked to complete the body of a function to do this.

In [ ]:
def scale_circuit(circuit: qiskit.QuantumCircuit, scale_factor: int) -> qiskit.QuantumCircuit:
    """Returns U (U^dag U)^k where U is the input circuit and k is the scale_factor.

    Args:
        circuit: Circuit to scale. Should not have measurements.
        scale_factor: The integer k in the above formula (how many times to add U^dag U).
    """
    # -->  Your code here.
    pass

One step down, and that's the hardest one!

### Step 2 / 3: Execute the noise-scaled computations

For this next step, we need to run our noise-scaled circuits. Because we defined an `execute` function above to compute our expectation value of interest, this is as easy as calling `execute` on each scaled circuit, as shown in the next cell.

In [ ]:
"""Pick scale factors and execute the computation at different noise levels."""
# Pick scale factors.
scale_factors = [1, 2, 3]

# Scale the circuits at each scale factor.
scaled_circuits = [scale_circuit(circuit, scale_factor) for scale_factor in scale_factors]

# Run the noise-scaled circuits.
expectation_values = [execute(scaled_circuit) for scaled_circuit in scaled_circuits]

In [ ]:
"""Plot the expectation value vs. scale factor data."""
# -->  Your code here.

### Step 3 / 3: Fit a model to the noise-scaled expectation values

If you kept the intended `depth` and `scale_factors`, the above trend line should look linear. So, let's fit a line $E(\lambda) = a + b \lambda$ to the data. We can do this very easily with `np.polyfit` as shown below.

In [ ]:
"""Fit a model to the data."""
best_fit_coeffs = np.polyfit(scale_factors, expectation_values, deg=1)

Now we have best-fit coefficients $E(\lambda) = a^* + b^* \lambda$. Use this to get the zero-noise $(\lambda = 0)$ value in the cell below.

In [ ]:
"""Get the `zne_value` from the best-fit coefficients found above."""
zne_value = 0.0  # <--  Your code here.

Congrats! you just implemented zero-noise extrapolation.

### Compare the results

Let's see how ZNE performed.

In [ ]:
"""Display the results."""
print("True: \t⟨𝜓|Π|𝜓⟩ =", true_value)
print("Noisy \t⟨𝜓|Π|𝜓⟩ =", noisy_value)
print("ZNE: \t⟨𝜓|Π|𝜓⟩ =", zne_value)

# Compute the "quantum error mitigation (QEM) factor."
error_mitigation_factor = (true_value - noisy_value) / (true_value - zne_value)
print("\nQEM factor:", error_mitigation_factor)

### To consider

- How do your results change if you fit a different degree polynomial?
- How do your results change if you use more / different `scale_factors`?
- How do your results change if you use a lower / higher depth circuit?
- Try fitting the data with an expononential or similar function using [SciPy curve fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).
- The fit you "should" use ideally comes from some knowledge of how the computation should look at different scale factors, based on both the details of the circuit, expectation value, and noise model of the device.

## Software for error mitigation

As we've (hopefully) seen, implementing zero-noise extrapolation isn't terribly difficult. But we just skimmed the surface - there's different ways to scale noise (including fractionally), choose scale factors (including adaptively), and fit models to the data. Error mitigation software packages like [Mitiq](https://mitiq.readthedocs.io/en/stable/) and [Qermit](https://cqcl.github.io/Qermit/) handle these details, as well as implement other error mitigation techniques. You may wish to use them for your own experiments instead of implementing everything "from scratch".

## Challenge

Done early?

- Try to implement zero-noise extrapolation for a quantum computation in one of the hands-on sessions at [this school](https://github.com/NuclearPhysicsWorkshops/FRIB-TASummerSchoolQuantumComputing#detailed-lecture-plan).
- Reproduce a zero-noise extrapolation experiment in literature, for example [this one increasing effective quantum volume](https://github.com/unitaryfund/mitiq-qv/blob/master/main.ipynb).
- Check out the other error mitigation methods in [Mitiq](https://mitiq.readthedocs.io/en/stable/) and/or [Qermit](https://cqcl.github.io/Qermit/).